In [1]:
import pandas as pd

In [3]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [4]:
HOTEL_BOOKINGS_DATASET_PATH = './datasets/hotel_bookings.csv'
hotel_bookings_df = read_data_from_csv(file_path=HOTEL_BOOKINGS_DATASET_PATH)

In [5]:
hotel_bookings_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


# Información sobre el dataset

## Atributos

| Variable  | Tipo de Variable  |  Descripción  |
|---|---|---|
| hotel  | Variable Cualitativa - Texto - Nominal  | Nombre del hotel  |
|  is_canceled | Variable Cualitativa - Texto - Nominal  |  Valor que indica si la reserva fue cancelada  |
|  lead_time | Variable Cuantitativa - Número - Discreta  | Cantidad de dias entre la reserva y la llegada al hotel  |
|  arrival_date_year | Variable Cuantitativa - Número - Discreta ? |  Año de arrivo |
| arrival_date_month  | Variable Cualitativa - Texto - Nominal ? |  Mes del arrivo |
| arrival_date_week_number  | Variable Cuantitativa - Número - Discreta ?  | Número de la semana del arrivo  |
|  arrival_date_day_of_month | Variable Cuantitativa - Número - Discreta ? | Día del mes del arrivo  |
|  stays_in_weekend_nights  | Variable Cuantitativa - Número - Discreta  |  Días del fin de semana (Sabado - Domingo) que abarca la reserva |
|  stays_in_week_nigths | Variable Cuantitativa - Número - Discreta  | Días de la semana (Lunes a Viernes) que abarca la reserva  |
|  adults | Variable Cuantitativa - Número - Discreta  | Cantidad de adultos  |
|  children | Variable Cuantitativa - Número - Discreta (En el Dataset aparece como continua)  |  Cantidad de niños. Cabe aclarar que los datos provenientes del dataset tiene tipo float y se debe transformar, ya que una reserva no puede tener 1.3 niños  |
| babies  | Variable Cuantitativa - Número - Discreta  | Cantidad de bebes  |
|  meal | Variable Cualitativa - Texto - Nominal  |  Tipo de comida reservada (Detalles en Categorias) |
|  country | Variable Cualitativa - Texto - Nominal  |  Pais de origen  |
| market_segment  | Variable Cualitativa - Texto - Nominal  | Segmento del mercado asignado (Detalles en Categorias)  |
| distribution_channel  |   |   |
|  is_repeated_guest | Variable Cualitativa - Número - Nominal  | Si el huesped ya se hospedo en el hotel  |
|  previous_cancelations |  Variable Cuantitativa - Número - Discreta |  Cantidad de cancelaciones  |
|  previous_bookings_not_canceled |  Variable Cuantitativa - Número - Discreta |  Cantidad de veces que no cancelo la reserva  |
| reserved_room_type  | Variable Cualitativa - Text - Nominal  | Código del tipo de cuarto reservado  |
| booking_changes  |  Variable Cuantitativa - Número - Discreta |  Cantidad de cambios en la reserva desde la fecha de reserva hasta la cancelación o check-in  |
| deposit_types  | Variable Cualitativa - Texto - Discreta  | Tipo de depósito que se hizo (Detalles en Categorias)   |
| agent  | Variable Cualitativa - Número - Nominal  | ID de la agencia que hizo la reserva   |
| company  |  Varaible Cualitativa - Número - Nominal | ID de la compania que hizo la reserva  |
| days_in_waiting_list  |  Variable Cuantitativa - Número - Discreta | Número de días que la reserva estuvo en lista de espera hasta ser confirmada  |
| customer_type  |  Variable Cualitativa - Texto - Nominal |  Tipo de reserva (Detalles en Categorias) |
|  adr |   |   |
|  required_car_parking_spaces |  Variable Cuantitativa - Número - Discreta | Cantidad de espacios de estacionamiento para el huesped  |
| total_of_special_requests  |  Variable Cuantitativa - Número - Discreta | Cantidad de pedidos especiales que hizo el huesped  |
|  reservation_status |  Variable Cualitativa - Texto - Nominal | Estado de la reserva (Detalles en Categorias)  |
|  reservation_status_date | Variable Cuantitativa - Número - Continua  | Fecha de la ultima vez que se actualizo reservation_status  |


## Categorias
- Meal:
    - Undefined/SC : No meal package.
    - BB : Bed & Breakfast.
    - HB : Breakfast and one other meal.
    - FB : Breafast, lunch and dinner.
- Market Segment:
    - TA : Agencias de viaje.
    - TO : Operador turístico.
- Deposit Type:
    - No Deposit : No se hizo un deposito.
    - Non Refund : Un deposito se hizo para el valor total de la reseva.
    - Refundable: Un deposito se hizo para un valor menor al total de la reserva.
- Customer Type:
    - Contract : Cuando la reserva esta asociada a un contrato.
    - Group : Cuando la reserva esta asocidada a un grupo.
    - Transient : Cuando la reserva no esta asociada a un contrato ni a un grupo.
    - Transient-party : Cuando la reserva es del tipo transient pero esta asociada a otra reserva transient.
- Reservation Status:
    - Canceled : La reserva fue cancelada.
    - Check-Out : El cliente hizo el check-in y tambien el check-out.
    - No-Show : El cliente no hizo el check-in pero tampoco cancelo la reserva.